In [37]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML
from reed_solomon_code.ReedSolomonCode import ReedSolomonCode

def rs_params_calc(m, t):
    n = pow(2, m) - 1
    k = n - 2 * t
    code_rate = round((k / n) * 100, 2)
    block_correction = round((t / n) * 100, 2)
    max_bits = n * k
    return [m, n, k, t, code_rate, block_correction, max_bits]

def rs_stat_test(solomon, poly_errors, parity_errors, is_decoding_success):
    """

    :type solomon: ReedSolomonCode
    """
    correct_probes = 0
    decoded_but_good = 0
    count = 1000
    for i in range(count):
        rand_message = ReedSolomonCode.generate_random_message(solomon.k, solomon.m)
        message = ReedSolomonCode.array_to_binary(rand_message, solomon.m)
        encoded_message = solomon.encode_number(message)
        mesage_with_errors = solomon.add_errors_string(poly_errors, encoded_message, is_parity=False)
        mesage_with_errors = solomon.add_errors_string(parity_errors, mesage_with_errors, is_parity=True)
        try:
            decoded_message = solomon.decode_number(mesage_with_errors)
            if decoded_message == ReedSolomonCode.remove_leading_zeros(message):
                correct_probes += 1
            else:
                decoded_but_good += 1

        except:
            continue
    if is_decoding_success:
        return (correct_probes / count) * 100
    return (decoded_but_good / count) * 100


In [38]:
data = np.array([rs_params_calc(4, 2), rs_params_calc(8, 8)])
columns = ["m", "n", "k", "t", "Współczynnik wiadomości [%]", 'Zdolność korekcyjna [%]', 'Maksymalna ilość bitów']
frame = pd.DataFrame(data=data, columns=columns)
frame = frame.astype({'m':'int','n':'int', 'k':'int', 't':'int', 'Maksymalna ilość bitów':'int'})
frame

,m,n,k,t,Współczynnik wiadomości [%],Zdolność korekcyjna [%],Maksymalna ilość bitów
0,4,15,11,2,73.33,13.33,165
1,8,255,239,8,93.73,3.14,60945


# Parametry
### m - liczba bitów na jakich zapisany jest pojedyńczy symbol
### n - liczba symboli w całym słowie
### k - liczba symboli do zakodowania jako wiadomośc w słowie
### t - liczba poprawialnych symboli
### Współczynnik wiadomości - stosunek liczby symboli do zakodowania w słowie do liczby wszystich symboli w słowie
### Zdolność korekcyjna - stosunek liczby poprawialnych symboli do liczby wszystkich symboli w słowie
### Maksymalna ilość bitów - ile maksymalnie bitów może pomieścić jedna wiadmość

In [39]:
solomon = ReedSolomonCode(4, 3)
arr = []
T_COLS = ['błędy w części wiadomości', 'błędy w części korekcyjnej', 'sukces odkodowania wiadomosci [%]', 'porażka wykrycia błędu [%]', 'stopień zepsucia wiadomości [%]']
for i in range(5):
    for j in range(5):
        arr.append([i, j, rs_stat_test(solomon, i, j, True), rs_stat_test(solomon, i, j, False), round(((i + j) / 15) * 100, 2)])

arr.append([9, 0, rs_stat_test(solomon, 9, 0, True), rs_stat_test(solomon, 9, 0, False), round((9 / 15) * 100, 2)])
arr.append([0, 6, rs_stat_test(solomon, 0, 6, True), rs_stat_test(solomon, 0, 6, False), round((6 / 15) * 100, 2)])
arr.append([9, 6, rs_stat_test(solomon, 9, 6, True), rs_stat_test(solomon, 9, 6, False), round((15 / 15) * 100, 2)])

frame = pd.DataFrame(data=arr, columns=T_COLS)
frame.sort_values(by=['sukces odkodowania wiadomosci [%]'], inplace=True, ascending=False)
display(HTML(frame.to_html(index=False)))


błędy w części wiadomości,błędy w części korekcyjnej,sukces odkodowania wiadomosci [%],porażka wykrycia błędu [%],stopień zepsucia wiadomości [%]
0,0,100.0,0.0,0.00
0,2,100.0,0.0,13.33
0,3,100.0,0.0,20.00
1,0,100.0,0.0,6.67
1,1,100.0,0.0,13.33
1,2,100.0,0.0,20.00
3,0,100.0,0.0,20.00
0,1,100.0,0.0,6.67
2,0,100.0,0.0,13.33
2,1,100.0,0.0,20.00
